In [25]:
import pandas as pd

In [26]:
import openai

with open("../openai_api_key.txt", "r") as f:
    api_key = f.read()

# openai.api_key = api_key

GPT_model = "gpt-3.5-turbo"

In [27]:
RANDOM_SEED = 123

client = openai.Client(api_key=api_key)

In [33]:
# read descriptions from json file
from data_load import load_games, load_system_prompt

games = load_games()
system_prompt = load_system_prompt('benchmark_prompt.txt')

In [34]:
games[0]

Name: A Simple Game
Description: This game is played on the following 4x4 square board. MOVE - On each turn, each player move one of his stones into an adjacent orthogonal empty cell. GOAL - Wins the player that makes a (orthogonal or diagonal) 3 in-a-row.
Code: (game "A Simple Game"
    (players 2)
    (equipment {
        (board (square 4) use:Vertex)
        (piece "Ball" Each ("StepToEmpty" Orthogonal))
        (hand Each)
    })
    (rules
        (start {
            (place "Ball1" (intersection (sites Phase 0) (union (sites Top) (sites Bottom))))
            (place "Ball2" (intersection (sites Phase 1) (union (sites Top) (sites Bottom))))
        })
        (play (forEach Piece))
        (end ("Line3Win" All))
    )
)

In [30]:
def get_model_response(message):
    completion = client.chat.completions.create(
    model=GPT_model,
    seed=RANDOM_SEED,
    messages=message,
    )
    return completion.choices[0].message.content

In [9]:
df = pd.read_csv('data/game_condition_query/asimplegame.csv')
df

,Unnamed: 0,task_id,language_full,language_phrase_1,language_phrase_2,language_phrase_3,language_phrase_4,language_phrase_5,language_phrase_6,language_phrase_7,...,code_phrase_1,code_phrase_2,code_phrase_3,code_phrase_4,code_phrase_5,code_phrase_6,code_phrase_7,code_phrase_8,question_code,answer
0,0,0,"Player 1 moves their piece from 1st row, 4th c...","Player 1 moves their piece from 1st row, 4th c...","Player 2 moves their piece from 1st row, 3rd c...","Player 1 moves their piece from 1st row, 2nd c...","Player 2 moves their piece from 4th row, 2nd c...","Player 1 moves their piece from 4th row, 3rd c...","Player 2 moves their piece from 4th row, 4th c...","Player 1 moves their piece from 3rd row, 3rd c...",...,(Move D4-D3),(Move C4-D4),(Move B4-B3),(Move B1-B2),(Move C1-C2),(Move D1-D2),(Move C2-C3),,,Player 1
1,1,1,"Player 1 moves their piece from 4th row, 3rd c...","Player 1 moves their piece from 4th row, 3rd c...","Player 2 moves their piece from 1st row, 3rd c...","Player 1 moves their piece from 1st row, 2nd c...",,,,,...,(Move C1-C2),(Move C4-C3),(Move B4-C4),,,,,,(Move C3-C2),An invalid move
2,2,2,"Player 1 moves their piece from 4th row, 3rd c...","Player 1 moves their piece from 4th row, 3rd c...","Player 2 moves their piece from 4th row, 2nd c...","Player 1 moves their piece from 4th row, 1st c...",,,,,...,(Move C1-C2),(Move B1-B2),(Move A1-B1),,,,,,(Move C4-C3),A valid move
3,3,3,"Player 1 moves their piece from 4th row, 3rd c...","Player 1 moves their piece from 4th row, 3rd c...","Player 2 moves their piece from 1st row, 3rd c...","Player 1 moves their piece from 1st row, 4th c...","Player 2 moves their piece from 2nd row, 3rd c...",,,,...,(Move C1-C2),(Move C4-C3),(Move D4-C4),(Move C3-B3),,,,,,No one


In [11]:
df = df[df.columns[1:]]

In [12]:
df

,task_id,language_full,language_phrase_1,language_phrase_2,language_phrase_3,language_phrase_4,language_phrase_5,language_phrase_6,language_phrase_7,language_phrase_8,...,code_phrase_1,code_phrase_2,code_phrase_3,code_phrase_4,code_phrase_5,code_phrase_6,code_phrase_7,code_phrase_8,question_code,answer
0,0,"Player 1 moves their piece from 1st row, 4th c...","Player 1 moves their piece from 1st row, 4th c...","Player 2 moves their piece from 1st row, 3rd c...","Player 1 moves their piece from 1st row, 2nd c...","Player 2 moves their piece from 4th row, 2nd c...","Player 1 moves their piece from 4th row, 3rd c...","Player 2 moves their piece from 4th row, 4th c...","Player 1 moves their piece from 3rd row, 3rd c...",,...,(Move D4-D3),(Move C4-D4),(Move B4-B3),(Move B1-B2),(Move C1-C2),(Move D1-D2),(Move C2-C3),,,Player 1
1,1,"Player 1 moves their piece from 4th row, 3rd c...","Player 1 moves their piece from 4th row, 3rd c...","Player 2 moves their piece from 1st row, 3rd c...","Player 1 moves their piece from 1st row, 2nd c...",,,,,,...,(Move C1-C2),(Move C4-C3),(Move B4-C4),,,,,,(Move C3-C2),An invalid move
2,2,"Player 1 moves their piece from 4th row, 3rd c...","Player 1 moves their piece from 4th row, 3rd c...","Player 2 moves their piece from 4th row, 2nd c...","Player 1 moves their piece from 4th row, 1st c...",,,,,,...,(Move C1-C2),(Move B1-B2),(Move A1-B1),,,,,,(Move C4-C3),A valid move
3,3,"Player 1 moves their piece from 4th row, 3rd c...","Player 1 moves their piece from 4th row, 3rd c...","Player 2 moves their piece from 1st row, 3rd c...","Player 1 moves their piece from 1st row, 4th c...","Player 2 moves their piece from 2nd row, 3rd c...",,,,,...,(Move C1-C2),(Move C4-C3),(Move D4-C4),(Move C3-B3),,,,,,No one


In [15]:
n_phrases_language = len(list(filter(lambda col_name: "language_phrase_" in col_name, df.columns)))
n_phrases_code = len(list(filter(lambda col_name: "code_phrase_" in col_name, df.columns)))
assert(n_phrases_language == n_phrases_code)
N_PHRASES = n_phrases_language
print(N_PHRASES)

8


In [35]:
game = games[0]
print(game.code_file_path)
path_name = game.code_file_path.split('/')[-1]
game_tag = path_name[:-4]
df = pd.read_csv(f'data/game_condition_query/{game_tag}.csv')
df = df[df.columns[1:]]
n_phrases_language = len(list(filter(lambda col_name: "language_phrase_" in col_name, df.columns)))
N_PHRASES = n_phrases_language

model_response = []
for _, row in df.iterrows():
    messages = []
    messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": game.description})
    for i in range(1, N_PHRASES + 1):
        if row[f"language_phrase_{i}"]:
            messages.append({"role": "user", "content": row[f"language_phrase_{i}"]})
    messages.append({"role": "user", "content": row["question_sentence"]})
    model_response.append(get_model_response(messages))

output_df = df[['task_id', 'answer']].copy()
output_df.assign(model_answer = model_response)
output_df.to_csv(f'data/benchmark_output/{game_tag}.csv')

AttributeError: 'GameModel' object has no attribute 'code_file_path'